In [ ]:
#Shubh Sudan
#21070126089
#AIML B1

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, RepeatVector, TimeDistributed,Input
from tensorflow.keras.models import Model
from tensorflow.keras.losses import sparse_categorical_crossentropy
import pandas as pd
import re
import string
from string import digits
import numpy as np
import warnings
warnings.filterwarnings("ignore")

2023-09-25 14:23:21.250060: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-25 14:23:21.303736: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 14:23:22.218971: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/student/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_m

In [4]:
df = pd.read_csv(r"Hindi_English_Truncated_Corpus (1).csv")
df.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [5]:
df.shape

(127607, 3)

In [6]:
df["source"].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [7]:
# Filtering out the sentence whose length is > 30.
data = df[(df.english_sentence.apply(lambda x:len(str(x)) <=30)) &
            (df.hindi_sentence.apply(lambda x:len(str(x)) <=30))].reset_index(drop=True)

In [8]:
data.shape

(18416, 3)

In [9]:
#Changing uppercase to lowercase
data["english_sentence"] = data["english_sentence"].apply(lambda x: str(x).lower()).copy()
data["hindi_sentence"] = data["hindi_sentence"].apply(lambda x: str(x).lower()).copy()

In [10]:
# Remove quotes
data["english_sentence"] = data["english_sentence"].apply(lambda x: re.sub("'",'',x))
data["hindi_sentence"] = data["hindi_sentence"].apply(lambda x: re.sub("'",'',x))

In [11]:
data.head()

,source,english_sentence,hindi_sentence
0,indic2012,category: religious text,श्रेणी:धर्मग्रन्थ
1,ted,this changed slowly,धीरे धीरे ये सब बदला
2,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
3,indic2012,maine,मेन
4,ted,can you imagine saying that?,क्या आप ये कल्पना कर सकते है


In [12]:
punctuation = set(string.punctuation)
data["english_sentence"] = data["english_sentence"].apply(lambda x: ''.join(ch for ch in x if ch not in punctuation))
data["hindi_sentence"] = data["hindi_sentence"].apply(lambda x: ''.join(ch for ch in x if ch not in punctuation))

In [13]:
data.head()

,source,english_sentence,hindi_sentence
0,indic2012,category religious text,श्रेणीधर्मग्रन्थ
1,ted,this changed slowly,धीरे धीरे ये सब बदला
2,ted,were being produced,उत्पन्न नहीं कि जाती थी
3,indic2012,maine,मेन
4,ted,can you imagine saying that,क्या आप ये कल्पना कर सकते है


In [14]:
# Remove digits
english = set(digits)
hindi =  	set(['०','१','२','३','४','५','६','७','८','९'])
digits = english.union(hindi)
data["english_sentence"] = data["english_sentence"].apply(lambda x: ''.join(ch for ch in x if ch not in digits))
data["hindi_sentence"] = data["hindi_sentence"].apply(lambda x: ''.join(ch for ch in x if ch not in digits))
data.tail(10)

,source,english_sentence,hindi_sentence
18406,indic2012,praja bharat,प्रजा भारत
18407,indic2012,asamiya,असमिया
18408,tides,fluoride,फ्ल्युराइड
18409,indic2012,surdas on wikipedia,सूरदास विकिस्रोत पर
18410,indic2012,hindustan samacharunicoded,हिन्दुस्थान समाचार यूनिकोडित
18411,indic2012,google author,गूगल अर्थ
18412,tides,the die was cast,सांचा ढल गया
18413,indic2012,devotional period,भक्ति काल
18414,indic2012,travelling,यात्राएं
18415,ted,thank you,धन्यवाद


In [15]:
# Removing extra space
data["english_sentence"] = data["english_sentence"].apply(lambda x: x.strip())
data["hindi_sentence"] = data["hindi_sentence"].apply(lambda x: x.strip())

data["english_sentence"] = data["english_sentence"].apply(lambda x: re.sub(" +"," ",x))
data["hindi_sentence"] = data["hindi_sentence"].apply(lambda x: re.sub(" +"," ",x))

In [16]:
data

,source,english_sentence,hindi_sentence
0,indic2012,category religious text,श्रेणीधर्मग्रन्थ
1,ted,this changed slowly,धीरे धीरे ये सब बदला
2,ted,were being produced,उत्पन्न नहीं कि जाती थी
3,indic2012,maine,मेन
4,ted,can you imagine saying that,क्या आप ये कल्पना कर सकते है
...,...,...,...
18411,indic2012,google author,गूगल अर्थ
18412,tides,the die was cast,सांचा ढल गया
18413,indic2012,devotional period,भक्ति काल
18414,indic2012,travelling,यात्राएं


In [17]:
input_text = []
target_text = []
input_characters = set()
target_characters = set()

for eng,hin in zip(data.english_sentence,data.hindi_sentence):
  target = "START_ " + hin + " _END"
  input_text.append(eng)
  target_text.append(target)

  for eng_char in eng.split():
    if eng_char not in input_characters:
      input_characters.add(eng_char)

  for hin_char in hin.split():
    if hin_char not in input_characters:
      target_characters.add(hin_char)

In [18]:
print(f"input text {len(input_text)}")
print(f"target text {len(target_text)}")
print(f"input characters  {len(input_characters)}")
print(f"output characters {len(target_characters)}")

input text 18416
target text 18416
input characters  9232
output characters 8575


In [19]:
print("Input Text ->>>>> "+ input_text[0]+ " ->>>>Output Text ->>>>>"+target_text[0])

Input Text ->>>>> category religious text ->>>>Output Text ->>>>>START_ श्रेणीधर्मग्रन्थ _END


In [20]:
input_char = sorted(list(input_characters))
target_char = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters) + 1 # 1 is for START_ AND _END

max_encoder_seq_length = max([len(txt) for txt in input_text])
max_decoder_seq_length = max([len(txt) for txt in target_text])

In [21]:
print("Number of samples:",len(input_text))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for output:", max_decoder_seq_length)

Number of samples: 18416
Number of unique input tokens: 9232
Number of unique output tokens: 8576
Max sequence length for inputs: 30
Max sequence length for output: 42


In [22]:
# CREATING INDEX FOR EACH AND EVERY WORD AND STORING AS DICTIONARY
input_token_index = dict([(word,i+1) for i, word in enumerate(input_char)])
target_token_index = dict([(word,i+1) for i, word in enumerate(target_char)])

reverse_input_char_index = dict([(i,word) for word, i in input_token_index.items()])
reverse_target_char_index = dict([(i,word) for word, i in target_token_index.items()])

In [23]:
print(input_token_index)

{'a': 1, 'aaaahhh': 2, 'aadikavya': 3, 'aagra': 4, 'aaj': 5, 'aajamgarth': 6, 'aakrnsa': 7, 'aakul': 8, 'aalekh': 9, 'aambat': 10, 'aangaare': 11, 'aanre': 12, 'aanri': 13, 'aarti': 14, 'aarun': 15, 'aaydaho': 16, 'aayouva': 17, 'aazamgad': 18, 'ab': 19, 'abandoned': 20, 'abdictions': 21, 'abhilekh': 22, 'abhinav': 23, 'abhishek': 24, 'abhyagat': 25, 'abidjan': 26, 'ability': 27, 'abinav': 28, 'abject': 29, 'able': 30, 'about': 31, 'about”': 32, 'above': 33, 'abraham': 34, 'absence': 35, 'absent': 36, 'absolute': 37, 'absolutely': 38, 'absorb': 39, 'abstract': 40, 'absurd': 41, 'abu': 42, 'abuse': 43, 'abusive': 44, 'abyogitha': 45, 'academic': 46, 'academy': 47, 'acceleration': 48, 'accept': 49, 'acceptable': 50, 'acceptance': 51, 'accepted': 52, 'access': 53, 'accessories': 54, 'acchyutan': 55, 'accident': 56, 'accidentally': 57, 'accolade': 58, 'accompanied': 59, 'accord': 60, 'according': 61, 'accordingly': 62, 'account': 63, 'accountability': 64, 'accounted': 65, 'accounts': 66, '

In [24]:
from sklearn.model_selection import train_test_split
X,y = data.english_sentence, data.hindi_sentence
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=2)
X_train.shape,X_test.shape

((16574,), (1842,))

In [25]:
# def generate_batch(X,y,batch_size):
#   while True:
#     for j in range(0,len(X),batch_size):
#       encoder_input_data = np.zeros((batch_size,max_encoder_seq_length),
#                                     dtype="float32")
#       decoder_input_data = np.zeros((batch_size,max_decoder_seq_length),
#                                     dtype="float32")
#       decoder_target_data = np.zeros((batch_size,max_decoder_seq_length,num_decoder_tokens),
#                                      dtype="float32")

#       for i,(input_text,target_text) in enumerate(zip(X[j:j+batch_size],y[j:j+batch_size])):
#         for t, word in enumerate(input_text.split()):
#           encoder_input_data[i,t] = input_token_index[word]

#         for t,word in enumerate(target_text.split()):
#           if t<len(target_text.split())-1:
#             decoder_input_data[i,t] = target_token_index[word]
#           if t>0:
#             decoder_target_data[i,t-1,target_token_index[word]] = 1.

#         yield([encoder_input_data,decoder_input_data],decoder_target_data)

    
    
# def generate_batch(X, y, batch_size):
#     while True:
#         for j in range(0, len(X), batch_size):
#             encoder_input_data = np.zeros((batch_size, max_encoder_seq_length), dtype="float32")
#             decoder_input_data = np.zeros((batch_size, max_decoder_seq_length), dtype="float32")
#             decoder_target_data = np.zeros((batch_size, max_decoder_seq_length, num_decoder_tokens), dtype="float32")

#             for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
#                 for t, word in enumerate(input_text.split()):
#                     if t < max_encoder_seq_length and word in input_token_index:
#                         encoder_input_data[i, t] = input_token_index[word]

#                 for t, word in enumerate(target_text.split()):
#                     if t < max_decoder_seq_length and word in target_token_index:
#                         decoder_input_data[i, t] = target_token_index[word]
#                     if t > 0 and word in target_token_index:
#                         decoder_target_data[i, t-1, target_token_index[word]] = 1.

#             yield ([encoder_input_data, decoder_input_data], decoder_target_data)


def generate_batch(X, y, batch_size):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_encoder_seq_length), dtype="float32")
            decoder_input_data = np.zeros((batch_size, max_decoder_seq_length), dtype="float32")
            decoder_target_data = np.zeros((batch_size, max_decoder_seq_length, num_decoder_tokens), dtype="float32")

            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    if t < max_encoder_seq_length and word in input_token_index:
                        encoder_input_data[i, t] = input_token_index[word]

                for t, word in enumerate(target_text.split()):
                    if t < max_decoder_seq_length and word in target_token_index:
                        decoder_input_data[i, t] = target_token_index[word]
                    if t > 0 and word in target_token_index:
                        decoder_target_data[i, t-1, target_token_index[word]] = 1.

            yield ([encoder_input_data, decoder_input_data], decoder_target_data)


In [38]:
latent_dim = 150

In [39]:
# Encoder
encoder_inputs = Input(shape = (None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim,return_state = True)
encoder_outputs,state_h,state_c = encoder_lstm(enc_emb)
encoder_states = [state_h,state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens,latent_dim,mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_ = decoder_lstm(dec_emb,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [40]:
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["acc"])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, None, 150)            1384800   ['input_5[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, None, 150)            1286400   ['input_6[0][0]']             
                                                                                            

In [41]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 20

model.fit_generator(generator = generate_batch(X_train,y_train,32),
                    steps_per_epoch = train_samples//batch_size,epochs=epochs,
                    validation_data = generate_batch(X_test,y_test,batch_size=batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/20
258/258 [==============================] - 74s 257ms/step - loss: 5.3076 - acc: 0.0343 - val_loss: 5.1598 - val_acc: 0.0408
Epoch 2/20
258/258 [==============================] - 61s 235ms/step - loss: 5.0181 - acc: 0.0385 - val_loss: 5.0442 - val_acc: 0.0442
Epoch 3/20
258/258 [==============================] - 51s 195ms/step - loss: 4.7372 - acc: 0.0410 - val_loss: 5.0397 - val_acc: 0.0448
Epoch 4/20
258/258 [==============================] - 60s 235ms/step - loss: 4.6275 - acc: 0.0423 - val_loss: 5.0259 - val_acc: 0.0492
Epoch 5/20
258/258 [==============================] - 51s 199ms/step - loss: 4.5674 - acc: 0.0468 - val_loss: 5.0062 - val_acc: 0.0499
Epoch 6/20
258/258 [==============================] - 60s 230ms/step - loss: 4.4757 - acc: 0.0482 - val_loss: 5.0296 - val_acc: 0.0549
Epoch 7/20
258/258 [==============================] - 51s 198ms/step - loss: 4.4215 - acc: 0.0513 - val_loss: 4.9856 - val_acc: 0.0556
Epoch 8/20
258/258 [==============================] - 6

In [43]:
model.save_weights('nmt_eng_hin_translation_trial_1.h5')

In [44]:
encoder_model = Model(encoder_inputs,encoder_states)

In [45]:
# Decoder Setup
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]

In [46]:
dec_emb2 = dec_emb_layer(decoder_inputs)

In [47]:
# To predict the next word in the sequence, set the initial states to the states from the previous t
decoder_outputs2,state_h2,state_c2 = decoder_lstm(dec_emb2,initial_state=decoder_states_inputs)
decoder_states2 = [state_h2,state_c2]
decoder_outputs = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist.

# final decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs2] + decoder_states2)

In [75]:
# def decode_sequence(input_seq):
#   # Encode the input as state vector
#   states_value = encoder_model.predict(input_seq)
#   target_seq = np.zeros(1,1)
#   # Populate the first character of target sequence with the staart character
#   # Target_seq[0,0] = target_token_index["START_"]
#   # Sampling loop for a batch of sequences
#   # (to simplify, here we asume a batch of size 1).
#   stop_condition = False
#   decode_sentence = ''
#   while not stop_condition:
#     output_tokens,h,c = decoder_model.predict([target_seq]+states_value)

#     # Sample a token
#     sampled_token_index = np.argmax(output_tokens[0,-1,:])
#     sampled_char = reverse_target_char_index[sampled_token_index]
#     decoded_sentence+=""+sampled_char

#     # Exit condition: either hit max length

#     # or find stop character
#     if(sampled_char == "_END" or len(decoded_sentence)>25):
#       stop_condition = True
#       # Update the target sequence (of length 1)/
#       target_seq = np.zeros((1,1))
#       target_seq[0,0] = sampled_token_index

#       # Update states
#       states_value = [h,c]
#   return decoded_sentence






def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))
    
    # Populate the first character of target sequence with the start character
    #target_seq[0, 0] = target_token_index['START_']
    
    # Sampling loop for a batch of sequences (simplified for batch size of 1)
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        # Map the token index to the corresponding character
        sampled_char = reverse_target_char_index[sampled_token_index]

        # Exit condition: either hit max length or find stop character
        if sampled_char == '_END' or len(decoded_sentence) > 25:
            stop_condition = True
        else:
            decoded_sentence += sampled_char

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence



In [76]:
val_gen = generate_batch(X_test,y_test,batch_size=1)
k=-1

In [77]:
k+=2
(input_seq,actual_output),_ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print("Input English sentence:", X_test[k:k+1].values[0])
print("Actual Hindi Translation:", y_test[k:k+1].values[0])
print("predicted Hindi Translation:",decoded_sentence[:])

1/1 [==============================] - 0s 34ms/step


KeyError: 0